# <center> TSA Account Closure First Notification</center>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID c18f9bf1-54ac-4057-a0e5-a7996475f33c successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-01-21,518944


The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [3]:
sq = """with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
)
,
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-01-21'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
)
,
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select 
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '2023-11-23')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-11-23')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and TSA_Opening_date < date '2023-07-26'  and (Last_TSA_Credit_Date < date '2023-07-26' and Last_TSA_Debit_Date < date '2023-07-26' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-07-26')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and tsa_status = 'Active'
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '2024-01-21'###DATE change here to D-1
)
"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(f"The count of rows and columns in the extracted data are\t:{df.shape}")


Job ID 168c90af-3c41-49cb-be8a-adeea4fc7f1c successfully executed: |          |
Downloading: 100%|██████████|
The count of rows and columns in the extracted data are	:(449547, 32)


In [6]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
100501,2010074,2023-04-23,60820100740001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2023-04-23,NaT,NaT,NaN,2024-01-21,Batch_1,January,0,0,NaT,NaT,NO ACTION
379549,1701937,2022-10-14,60817019370007,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2022-10-14,NaT,NaT,NaN,2024-01-21,Batch_1,January,0,0,NaT,NaT,NO ACTION
168983,2219618,2023-09-03,60822196180009,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2023-09-04,NaT,NaT,NaN,2024-01-21,Batch_1,January,1,0,2023-10-04,NaT,NO ACTION
59709,2036008,2023-05-09,60820360080005,0.0,Active,3.98,0.0,0.0,0.0,0,2023-11-30,2023-11-30,N,0,0.0,1,0,0,0,0,2023-09-15,2024-01-21,NaT,NaN,2024-01-21,Batch_1,January,0,0,2023-11-15,NaT,NO ACTION
78435,1281886,2022-03-21,60812818860003,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2022-10-19,NaT,NaT,NaN,2024-01-21,Batch_1,January,1,0,2022-11-19,NaT,NO ACTION
305894,1037943,2021-05-14,60810379430002,0.0,Active,0.00,0.0,0.0,0.0,0,2023-04-10,2023-04-10,N,0,0.0,0,1,0,0,0,2023-04-10,2022-09-25,NaT,NaN,2024-01-21,Batch_1,January,1,0,2022-08-03,NaT,NO ACTION
249572,1899432,2023-02-13,60818994320001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2023-02-13,NaT,NaT,NaN,2024-01-21,Batch_1,January,1,0,2023-03-16,NaT,NO ACTION
312276,1607831,2022-09-09,60816078310006,0.0,Active,60.62,0.0,0.0,0.0,0,2023-12-31,2023-12-31,N,0,0.0,0,0,0,0,0,2022-11-11,2024-01-21,NaT,NaN,2024-01-21,Batch_1,January,0,0,NaT,NaT,NO ACTION
362744,1963465,2023-03-27,60819634650006,0.0,Active,2.09,43.7,0.0,0.0,0,2024-01-01,2024-01-01,N,1,37974.1,1,0,0,0,0,2024-01-07,2024-01-21,NaT,NaN,2024-01-21,Batch_1,January,0,0,NaT,NaT,NO ACTION
195047,1080331,2021-08-27,60810803310007,0.0,Active,0.00,0.0,0.0,0.0,0,2023-08-31,2023-09-20,N,1,0.0,1,1,0,0,0,2023-09-11,2023-09-19,NaT,NaN,2024-01-21,Batch_1,January,0,0,NaT,NaT,NO ACTION


In [7]:
df.to_excel(r"D:\MyStuff\Marvin\TSA Account Closure\Dec23 Batch & Jan24 Ongoing Batch\20240122_TSA_Account_Closure_1stNotification.xlsx", sheet_name='DATA', index = False)